In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import io
import pandas as pd
import random
import scipy.io as sio
from scipy import ndimage
import scipy
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import torch
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision import models

import torch
import torch.nn as nn
import math
from functools import partial
from torch.autograd import Variable

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

import time
import datetime
import copy

ModuleNotFoundError: No module named 'torch'

In [ ]:
#read excel file
BASE_DIR='.\MI_Data'
data_df = pd.read_excel(".\MI_polyps.xlsx")
data_df.head()
#print (data_df)


In [ ]:
class Polyps_Dataset(Dataset):
    def __init__(self,df:pd.DataFrame,imfolder:str,train:bool = True, transforms=None):
        self.df=df
        self.imfolder=imfolder
        self.train=train
        #self.transforms=transforms
        
    def __getitem__(self,index):
        filename=self.df.iloc[index]['fp'];
        
        im_path=os.path.join(self.imfolder,filename.strip())
        data = sio.loadmat(im_path)
        x = torch.from_numpy(data['xstack'])
        sz=x.size()
        if(len(sz)<3):
            x=x.unsqueeze(2)
            
        x=x.permute(2,0,1)
        #print(x.size())
       
        if(self.train):
            y=self.df.iloc[index]['Class']
            return x,y
        else:
            return x
        
    def __len__(self):
        return len(self.df)

In [ ]:
train, valid = train_test_split(
    data_df, 
    test_size=0.2, 
    random_state=42,
    stratify=data_df.Class
)

def my_collate(batch):
    data=torch.tensor([])
    target=torch.tensor([])
    pat=torch.tensor([])
    
    for index, item in enumerate(batch):
        sz=item[0].size()
        data=torch.cat((data,item[0]),dim=0)
        
        lab=torch.ones(sz[0],)*item[1].astype(int) 
        target=torch.cat((target,lab),dim=0)
        
        inst=torch.ones(sz[0],)*index 
        pat=torch.cat((pat,inst),dim=0)
    
    data=data.unsqueeze(1)
    return [data, target, pat]

# reset index on both dataframes
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

train_targets = train.Class.values

# targets for validation
valid_targets = valid.Class.values

train_dataset=Polyps_Dataset(
    df=train,
    imfolder=BASE_DIR,
    train=True,
    transforms=None
)

valid_dataset=Polyps_Dataset(
    df=valid,
    imfolder=BASE_DIR,
    train=True,
    transforms=None
)

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    #num_workers=4,
    shuffle=True,
    collate_fn=my_collate,
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=1,
    #num_workers=4,
    shuffle=False,
    collate_fn=my_collate,
)

In [ ]:
inputs, targets, instances = next(iter(train_loader))